In [9]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

In [10]:
# Load data set for bugs and sentiments

print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)

/home/cr/Desktop/Implementation/bugs
-----------------
['Core.json', 'Toolkit.json', 'bugzilla.json', 'Calendar.json', 'SeaMonkey.json', 'Camino Graveyard.json', 'MailNews Core.json', 'Firefox.json', 'Thunderbird.json', 'Core Graveyard.json']


In [150]:
# Load data for bugs 
train_set = []
test_set = []

for count, report in enumerate(BUG_REPORTS_LIST):
    if count ==3:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            test_set.extend(temp_data["bugs"])
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            train_set.extend(temp_data["bugs"])   

Train Set: Core.json
File:Core.json <> Reports: 7228
Train Set: Toolkit.json
File:Toolkit.json <> Reports: 1899
Train Set: bugzilla.json
File:bugzilla.json <> Reports: 5324
Test Set: Calendar.json
File:Calendar.json <> Reports: 1785
Train Set: SeaMonkey.json
File:SeaMonkey.json <> Reports: 8716
Train Set: Camino Graveyard.json
File:Camino Graveyard.json <> Reports: 1290
Train Set: MailNews Core.json
File:MailNews Core.json <> Reports: 2796
Train Set: Firefox.json
File:Firefox.json <> Reports: 7368
Train Set: Thunderbird.json
File:Thunderbird.json <> Reports: 5129
Train Set: Core Graveyard.json
File:Core Graveyard.json <> Reports: 888


In [151]:
summary_train=[]
label_train=[]
summary_test=[]
label_test=[]
for i in train_set:
    summary_train.append(i['summary'])
    #summary_train=summary_train[0:10]
    label_train.append(i['resolution'])
print('train_summary',len(summary_train))
print('train_label',len(label_train))

    #label_train=label_train[0:10]
for j in test_set:
    summary_test.append(j['summary'])
    #summary_test=summary_test[0:10]
    label_test.append(j['resolution'])
   # label_test=label_test[0:10]
print('test_summary',len(summary_test))
print('test_label',len(label_test))

train_summary 40638
train_label 40638
test_summary 1785
test_label 1785


In [152]:
#Preprocess sentences
import re
def train_test_summary(summaries_text):
    
    lemmas=[]
    lemmatized_word=[]
    train_test_summaries=[]
    for count, summaries in enumerate (summaries_text):
        text = re.sub(r'[^a-zA-Z\']', ' ', summaries)
        #removes all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        #substitute multiple spcaces with single space
        s_m_s = re.sub(r'\s+', ' ', document, flags=re.I)
        #removes prefix
        r_prefix = re.sub(r'^b\s+', '', s_m_s)
        #lemmas.append(r_prefix)
        r_pm = re.sub(r'[^\w\s]', '', r_prefix)
        step1 = r_pm.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
        step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
        step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
        step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
        step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
        step6 = step5.replace(" ` ", " '")
        step7=step6.strip()
        step8=step7.lower()    
        lemmas=[]
        sentences = nltk.word_tokenize(step7)
        lemmatizer = WordNetLemmatizer()
        stem_words=[]
        for sentence_words in sentences:
            lower=sentence_words.lower()
            if lower not in stopwords.words('english'):
                lemma = lemmatizer.lemmatize(lower)
                stem_words.append(lemma)
            untokenized_sentence= ' '.join(stem_words)
        train_test_summaries.append(untokenized_sentence)
    return train_test_summaries

In [153]:
# Encocede Labels 
def train_test_labels(labels):
    train_test_label=[]
    for x in range(len(labels)):
        
        
        if label_train[x]=="FIXED":
            label_train[x]=1
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="INVALID":
            label_train[x]=0
        elif label_train[x]=="INCOMPLETE":
            label_train[x]=0
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="WORKSFORME":
            label_train[x]=0
        elif label_train[x]=="EXPIRED":
            label_train[x]=0
        elif label_train[x]=="DUPLICATE":
            label_train[x]=0
        elif label_train[x]=="":
            label_train[x]=0
        train_test_label.append(label_train[x])
        
    
    
    #print('train_lbel', len(train_label))
    return train_test_label

In [154]:
train_labels= train_test_labels(label_train)
test_labels=train_test_labels(label_test)

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)


print(f"Training Labels Length:{len(train_labels)}")
print(f"Testing Labels Length:{len(test_labels)}")

print(f'Total len: {len(train_labels)+len(test_labels)}')

# Use 
#     train_labels
#     test_labels

Training Labels Length:40638
Testing Labels Length:1785
Total len: 42423


In [155]:
train_summary=train_test_summary(summary_train)
test_summary=train_test_summary(summary_test)

print(f"Training summary Length:{len(train_summary)}")
print(f"Testing summary Length:{len(test_summary)}")
print(f'Total len: {len(train_summary)+len(test_summary)}')



# Use 
#     train_summary
#     test_summary

Training summary Length:40638
Testing summary Length:1785
Total len: 42423


In [156]:
print(train_summary[1])

split alignment back three button


In [157]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer Transforms text into a sparse matrix of n-gram counts.

vectorizer= CountVectorizer(input='content', encoding='utf-8',
                lowercase=True, preprocessor=None,stop_words=stopwords.words('english'),
                ngram_range=(3, 3), analyzer='word', max_df=0.7, min_df=1, max_features= 8000, 
                binary=False)


In [158]:
#The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted.
#All values of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1)
#means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. 
#Only applies if analyzer is not callable.

In [159]:
print(train_summary[1])

split alignment back three button


In [160]:
train = vectorizer.fit_transform(summary_train)

In [161]:
vectorizer.get_feature_names()

['10 10 os',
 '1001 hangul syllables',
 '17 22 30',
 '2014 batch ev',
 '2015 batch ev',
 '22 30 34',
 '2369 list headers',
 '30 34 44',
 '32 links causes',
 '34 44 minimum',
 '3rd button menu',
 '3rd party cookie',
 '500 mhz computer',
 '64 bit builds',
 '64 bit integers',
 '64bit safe code',
 '_blank new tab',
 'aahig open dialog',
 'ab card pane',
 'ab enable category',
 'abilities user js',
 'ability add attachments',
 'ability add bookmark',
 'ability add buttons',
 'ability add custom',
 'ability add new',
 'ability add remove',
 'ability add testcase',
 'ability always show',
 'ability assign custom',
 'ability attach filters',
 'ability automatically save',
 'ability block flash',
 'ability block images',
 'ability bookmark mail',
 'ability browse mail',
 'ability change bookmark',
 'ability change order',
 'ability close tabs',
 'ability copy image',
 'ability create filters',
 'ability create forms',
 'ability create new',
 'ability delete search',
 'ability disable form',
 'a

In [162]:
test = vectorizer.transform(summary_test)

In [163]:
print(train[80].toarray())

[[0 0 0 ... 0 0 0]]


In [164]:
print(test.shape)
print(train.shape)

(1785, 8000)
(40638, 8000)


In [165]:
from sklearn import metrics

In [166]:
#1. MUltinomail_naive Bayes
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(train, train_labels)
predicted = clf.predict(test)


metrics.confusion_matrix(test_labels, predicted)
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(test_labels, predicted)


print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score



print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

Average precision-recall score: 0.27
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1308
           1       0.33      0.00      0.00       477

    accuracy                           0.73      1785
   macro avg       0.53      0.50      0.42      1785
weighted avg       0.63      0.73      0.62      1785

Accuracy: 0.7322128851540616
F1 score: 0.004166666666666667
Recall: 0.0020964360587002098
Precision: 0.3333333333333333
matthews_corrcoef: 0.006129509511525356

 confussion matrix:
 [[1306    2]
 [ 476    1]]


In [167]:
# 2. Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=6)
decision_tree = decision_tree.fit(train, train_labels)
predicted=decision_tree.predict(test)



print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1308
           1       0.00      0.00      0.00       477

    accuracy                           0.73      1785
   macro avg       0.37      0.50      0.42      1785
weighted avg       0.54      0.73      0.62      1785

Accuracy: 0.7327731092436974
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: 0.0

 confussion matrix:
 [[1308    0]
 [ 477    0]]


/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [169]:
# 3. SVM
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

X, y = make_classification(n_features=10, random_state=0)
clf = make_pipeline(StandardScaler(with_mean=False),
                     LinearSVC(random_state=0, tol=1e-5))

clf.fit(train, train_labels)
#print(clf.named_steps['linearsvc'].intercept_)
#print(clf.named_steps['linearsvc'].coef_)
predicted=clf.predict(test)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.73      0.99      0.84      1308
           1       0.22      0.00      0.01       477

    accuracy                           0.73      1785
   macro avg       0.48      0.50      0.43      1785
weighted avg       0.60      0.73      0.62      1785

Accuracy: 0.7299719887955182
F1 score: 0.00823045267489712
Recall: 0.0041928721174004195
Precision: 0.2222222222222222
matthews_corrcoef: -0.007239898878513887

 confussion matrix:
 [[1301    7]
 [ 475    2]]


/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [146]:
# 4. Bernouli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
clf_bn = BernoulliNB()
clf_bn.fit(train, train_labels)
predicted=clf_bn.predict(test)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1407
           1       0.00      0.00      0.00       492

    accuracy                           0.74      1899
   macro avg       0.37      0.50      0.42      1899
weighted avg       0.55      0.74      0.63      1899

Accuracy: 0.7388098999473407
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: -0.02716820159664945

 confussion matrix:
 [[1403    4]
 [ 492    0]]


In [147]:
#5.KNN 
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()  
model.fit(train, train_labels)

predicted=model.predict(test)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))


              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1407
           1       0.00      0.00      0.00       492

    accuracy                           0.74      1899
   macro avg       0.37      0.50      0.43      1899
weighted avg       0.55      0.74      0.63      1899

Accuracy: 0.7393364928909952
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: -0.023522147205107674

 confussion matrix:
 [[1404    3]
 [ 492    0]]


In [148]:
# 6. Logistic Regression 

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0).fit(train, train_labels)
predicted = clf_lr.predict(test)


print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1407
           1       0.00      0.00      0.00       492

    accuracy                           0.74      1899
   macro avg       0.37      0.50      0.43      1899
weighted avg       0.55      0.74      0.63      1899

Accuracy: 0.740916271721959
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: 0.0

 confussion matrix:
 [[1407    0]
 [ 492    0]]


/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [149]:
# 7. Mutlilayer_perceptron
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5 , hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(train, train_labels)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,solver='lbfgs')
predicted=clf.predict(test)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1407
           1       0.00      0.00      0.00       492

    accuracy                           0.74      1899
   macro avg       0.37      0.50      0.43      1899
weighted avg       0.55      0.74      0.63      1899

Accuracy: 0.740916271721959
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: 0.0

 confussion matrix:
 [[1407    0]
 [ 492    0]]


/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
"""Compute the Matthews correlation coefficient (MCC)

The Matthews correlation coefficient is used in machine learning as a measure of the quality of binary and multiclass classifications. It takes into account true and false positives and negatives and is generally regarded as a balanced measure which can be used even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction. The statistic is also known as the phi coefficient. [source: Wikipedia]

Binary and multiclass labels are supported. Only in the binary case does this relate to information about true and false positives and negatives. See references below."""